### 線形システムのEMアルゴリズム

#### EMアルゴリズムの理解のためには
* 動的システムのベイズ推論
* 線形ガウシアンシステムについて
* カルマンフィルターとRTSフィルタリング
* イエンセンの不等式
* MMアルゴリズム

などの理解が必要

#### 推論について

目的: 現在のシステムの状態を今までの観測から推測

* 状態空間モデルで表現されている。
* システムの状態の初期値に関しては事前知識として与えられる。
* 現在までの観測が存在

推論: これらを用いて現在の状態の事後分布を推定することがやりたいこと

#### ベイジアンフィルタリングの導出

\begin{align}
 p(x_t|y_{1:t})&=\frac{p(y_t|x_t, y_{1:t-1})\cdot p(x_t| y_{1:t-1})}{p(y_t|y_{1:t-1})} \\&= \alpha\cdot p(y_t|x_t, y_{1:t-1})\cdot p(x_t| y_{1:t-1})
\end{align}

$$
 p(y_t | x_t, y_{1:t-1}) = p(y_t | x_t)
$$

マルコフ過程なので$x_t$は十分統計量

\begin{align}
 p(x_t | y_{1:t-1}) &= \int{p(x_t, x_{t-1} | y_{1:t-1})} dx_{t-1} \\
                    &= \int{[p(x_t | x_{t-1}, y_{1:t-1})\cdot p(x_{t-1} | y_{1:t-1})]}dx_{t-1}\\
                    &= \int{[p(x_t | x_{t-1})\cdot p(x_{t-1} | y_{1:t-1})]}dx_{t-1}
\end{align}

変量$x_{t-1}$を導入したうえで周辺化を行う。これをベイズの規則とマルコフ過程の仮定により再帰的な更新式を得る。

結局、
$$
  p(x_t|y_{1:t}) = \alpha\cdot p(y_t|x_t)\cdot \int{[p(x_t | x_t-1)\cdot p(x_{t-1}| y_{1:t-1}]}dx_{t-1}
$$
という形により(事後分布)=(観測モデル)$\times$(観測モデルと事前分布の積の積分)として表される

$\alpha$は異常検知などに使える量であるが今回は無視

#### 予測とスムージング

予測とスムージングは似ているがフィルタリングとは異なる。
* フィルタリングは$p(x_t|y_{1:t})$を過去の観測から得るもの
* 予測は$p(x_{t+s}|y_{1:t})$を得るもの
* スムージングは全観測を既に得ている状態で過去の状態について推定するもの$p(x_t|y_{1:t+s})$

#### ベイズ予測
\begin{align}
p(x_{t+1}|y_{1:t})&=\int{p(x_{t+1},x_t|y_{1:t})}dx_t \\
                  &=\int{[p(x_{t+1}|x_t)\cdot p(x_t|y_{1:t})]}dx_t
\end{align}

* 1ステップ先を予測するには積分は一回
* kステップならk回
* 基本的には多重積分は難しいが、正規分布は簡単

#### スムージング
* 再帰的な後ろ向き推定式を得る

$$
p(x_t|y_{1:T})=\int{p(x_t, x_{t+1}|y_{1:T})}dx_{t+1}
$$

\begin{align}
p(x_t, x_{t+1}|y_{1:T}) &= p(x_t|x_{t+1}, y_{1:T})\cdot p(x_{t+1}|y_{1:T}) \\
                        &= p(x_t|x_{t+1}, y_{1:t})\cdot p(x_{t+1}|y_{1:T}) \\
                        &= \frac{p(x_t, x_{t+1}|y_{1:t})}{p(x_{t+1}|y_{1:t})}\cdot p(x_{t+1|y_{1:T}}) \\
                        &= \frac{p(x_{t+1}|x_t, y_{1:t})\cdot p(x_t|y_{1:t})\cdot p(x_{t+1}|y_{1:T})}{p(x_{t+1}|y_{1:t})}
\end{align}

よって積分により以下の式を得る。
$$
 p(x_t|y_{1:T})=\int{\frac{p(x_{t+1}|x_t)\cdot p(x_t|y_{1:t})\cdot p(x_{t+1}|y_{1:T})}{p(x_{t+1}|y_{1:t})}}dx_{t+1}
$$

#### 線形ガウシアンについて

$x,~y$は多変量ガウス分布とする。
$$
  x\sim \mathbf{N}(\mu_x, \Sigma_x),~y\sim \mathbf{N}(\mu_y, \Sigma_y)
$$

* 連結ガウシアンもガウシアン
$$
  \left[\begin{matrix}
    x\\
    y
  \end{matrix}\right]\sim \mathbf{N}\left(\left[\begin{matrix}
    \mu_x\\
    \mu_y
  \end{matrix}\right],\left[\begin{matrix}
    \Sigma_x & \Sigma_{xy}\\
    \Sigma_{yx} & \Sigma_y
  \end{matrix}\right]\right)
$$

$$
  \Sigma_x = \mathbf{V}(x) = \mathbf{E}[(x-\mu_x)(x-\mu_x)^T]
$$

$$
  \Sigma_{xy} = \mathbf{E}[(x-\mu_x)(y-\mu_y)^T]
$$

* ガウシアンの線型結合もガウシアン
$$
  x\sim \mathbf{N}(\mu_x, \Sigma_x)\\
  y\sim \mathbf{N}(\mu_y, \Sigma_y)\\
  cov(x, y)=0
$$

のとき

$$
  \mathbf{A}x+\mathbf{B}y\sim \mathbf{N}(\mathbf{A}\mu_x+\mathbf{B}\mu_y, \mathbf{A}\Sigma_x\mathbf{A}^T+\mathbf{B}\Sigma_y\mathbf{B}^T)
$$

$x,~y$が連結ガウシアンの時
$$
  p(y|x)=\mathbf{N}(y|\mu_y + \Sigma_{xy}^T{\Sigma_x}^{-1}(x-\mu_x), \Sigma_y-\Sigma_{xy}^T{\Sigma_x}^{-1}\Sigma_{xy})
$$

#### 線形ガウシアンシステムに関する重要な結果

$x$はガウシアンであり$x\sim N(m, P)$とする
$y$は$x$のアフィン変換に白色ノイズが乗っているものとする。

$$
 v\sim \mathbf{N}(0, R)\\
 y=\mathbf{A}x+b+v
$$